In [1]:
import pandas as pd

# 결측치 데이터
df_titanic = pd.read_csv('data/titanic.csv')
# 이상치 데이터
df_berry = pd.read_csv('data/wild_blueberry.csv')

In [ ]:
df_titanic.info()

In [ ]:
df_blueberry.info()

### 1. 결측치 처리

##### (1) 결측 레코드 제거
처리는 쉽지만 정보손실 동반

In [ ]:
df_titanic.dropna() # 이렇게 해서는 실제로 Dataframe에 반영은 안 됨
df_titanic.dropna(inplace=True) # 실제로 반영하기 위해서는 이처럼 inplace=True를 dropna에 전달하거나
df_titanic = df_titanic.dropna() # 이처럼 대입을 해야함

In [ ]:
df_titanic.dropna().isna().sum().to_frame().T

##### (2) 단순 대체
- 범주형 데이터

    결측 자체도 하나의 범주값으로 보고, 결측을 의미하는 범주를 생성하여 대체

##### Q1
Embarked의 결측치를 살펴보고 처리하기 데이터

In [ ]:
# Mine (Wrong)
df_titanic = df_titanic[df_titanic['Embarked'].isna()]['Embarked'].apply(
    lambda x: "Unknown"
)

In [ ]:
# Lectures'
df_titanic['Embarked'].unique()

In [ ]:
# pd.Series.fillna를 통해 결측치 대체 가능
df_titanic['Embarked'] = df_titanic['Embarked'].fillna("Unkonwn")

In [ ]:
df_titanic['Embarked'].value_counts() # value_counts 실행 시 na 값은 안 보임

- 최빈값으로 대체

##### Q2

Cabin은 복수의 요소로 구성이 되어 있고, 공백으로 구분되어 있습니다.

이 중 맨 처음 나오는 Cabin만 남깁니다.

앞자리 알파벳을 Cabin_Type, 그리고 뒷자리 숫자는 Cabin_No로 나누어 파생 변수를 만듭니다.

Cabin_Type을 Pclass 별로 Cabin_Type의 최빈값으로 대체합니다.

만일 동수가 등장할 경우 사전순으로 앞에 있는 것으로 대체합니다.

In [ ]:
# 이 중 맨 처음 나오는 Cabin만 남깁니다.
# 앞자리 알파벳을 Cabin_Type, 그리고 뒷자리 숫자는 Cabin_No로 나누어 파생 변수를 만듭니다.
df_titanic[['Cabin_Type', 'Cabin_No']] = df_titanic['Cabin'][df_titanic['Cabin'].notna()]\
    .apply(
        lambda x: x.split(' ')[0]
    )\
    .apply(
        lambda x: pd.Series([x[:1], x[1:]], index=['Cabin_Type', 'Cabin_No'])
    )

In [ ]:
df_titanic.groupby(['Pclass', 'Cabin_Type'])['Cabin_Type'].count()

In [ ]:
df_titanic['Cabin_Type'].mode().sort_values().iloc[0]

In [ ]:
df_titanic.groupby('Pclass')['Cabin_Type'].agg(
    lambda x: x.mode().sort_values().iloc[0]
)

In [ ]:
df_titanic['Cabin_Type'].fillna(
    df_titanic.groupby('Pclass')['Cabin_Type'].transform(
       lambda x: x.mode().sort_values().iloc[0]
    )
)

In [ ]:
# Cabin_Type을 Pclass 별로 Cabin_Type의 최빈값으로 대체합니다.
# 만일 동수가 등장할 경우 사전순으로 앞에 있는 것으로 대체합니다.
df_titanic['Cabin_Type'] = df_titanic.groupby('Pclass')['Cabin_Type'].transform(
    lambda x: x.fillna(x.mode().sort_values().iloc[0])
)

- 수치형 데이터

    i) 중심 통계량(평균, 중앙값, 최빈값)으로 대체

In [ ]:
df_titanic['Age'] = df_titanic['Age'].fillna(df_titanic['Age'].mean())


    ii) 보간법(Interpolation)
    
        순서가 있는 데이터(시간, 위치)의 경우 앞, 뒤 값을 활용해 결측치를 대체

In [ ]:
# 샘플 데이터를 만듭니다.
s = pd.Series([1, np.nan, np.nan, 7], index=[1, 2.5, 3, 4])

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
# 결측이 아닌 두점을 잇는 선을 그립니다.
# linear: x 값은 고려하지 않고, 등간격이라는 가정하에 선형 보간을 합니다.
# index: index의 값이 x값이라고 가정하고 선형 보간을 합니다.
for i, ax in zip(['linear', 'index'], axes):
    ax.plot(s.loc[s.notna()])

    # 결측이 아닌 값을 찍습니다.
    s.loc[s.notna()].pipe(
        lambda x: ax.scatter(x=x.index, y=x.values, color='red', label='not missing')
    )
    # 결측인 값을 보간한 값을 찍습니다.
    s.interpolate(method=i).loc[s.isna()].pipe(
        lambda x: ax.scatter(x=x.index, y=x.values, color='blue', label='interpolated')
    )

    ax.legend()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title("label=" + i)
plt.tight_layout()
plt.show()

- Regression

    다른 변수를 이용하여 결측치를 예측하는 모델을 만들어, 모델의 예측값으로 결측치을 대체합니다.

##### Q3
<div style="border: 1px solid #ddd; padding: 12px; margin-top: 10px;">

'Age'의 결측값을 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked' 를 이용하여 예측 모델을 만들어 예측치로 결측값을 대체합니다.

예측 모델은 Random Forest Regression 모델로 하고, Pclass, Sex, Embarked는 가변수화 합니다. 나머지 입력값은 그대로 모델에 사용합니다.

</div>

In [ ]:
# 전처리 단계의 출력을 바로 모델에 전달 될 수 있도록 make_pipeline을 사용하여 Pipeline을 구성토록 합니다.
from sklearn.pipeline import make_pipeline
# ColumnTranformer를 사용하여 변수별 전처리 과정을 설정할 수 있도록 합니다.
from sklearn.compose import ColumnTransformer
# 가변수 처리기를 가져옵니다.
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

# Embarked는 최빈값으로 결측을 처리합니다.
df_titanic['Embarked'] = df_titanic['Embarked'].fillna(df_titanic['Embarked'].mode().iloc[0])

# OneHotEncoder는 Pclass, Sex, Embarked
# SibSp, Parch, Fare는 처리 없이 통과 시킵니다.
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(), ['Pclass', 'Sex', 'Embarked']), # (이름, 처리기, 처리 대상 변수)
    ('pt', 'passthrough', ['SibSp', 'Parch', 'Fare'])
])

reg_rf = make_pipeline(
    ct,
    RandomForestRegressor(
        n_estimators=100, # RandomForest에서 사용할 의사 결정 나무의 수를 100개로 합니다.
        random_state=123) # random_state에 값을 지정하여 실행시 동일한 결과를 얻게 설정합니다.
)
X_rf = ['Pclass', 'Sex', 'Embarked', 'SibSp', 'Parch', 'Fare'] # 입력변수 리스트 입니다.
# Regression 토픽에 중심을 두겠습니다: 모델에 대한 검증은 생략하고 바로 학습을 합니다.

df_titanic.loc[df_titanic['Age'].notna()].pipe(
    lambda x: reg_rf.fit(x[X_rf],  x['Age'])
) # 모델을 학습시킵니다.
"""
위 루틴과 동일한 루틴입니다. 
위 구문은 아래 구문에 df_titaninc['Age'].notna()로 필터링 2회 했던 것을 pipe를 통해 줄여낸 것입니다.
reg_rf.fit(
    df_titanic.loc[df_titanic['Age'].notna(), X_rf], 
    df_titanic.loc[df_titanic['Age'].notna(),'Age']
) 
"""

# 결측값에 예측 결과를 적용합니다.
df_titanic.loc[df_titanic['Age'].isna(), 'Age'] = \
    reg_rf.predict(df_titanic.loc[df_titanic['Age'].isna(), X_rf])

## 2. 이상치 처리
IQR(Inter Quantile Range)을 통한 이상치 판별

- 이상치 제거

    이상치로 판단한 데이터를 제거

##### Q4

<div style="border: 1px solid #ddd; padding: 12px; margin-top: 10px;">

df_berry 셋의 fruitset, fruitmass, seeds 가

각 변수별로 $Q_1$, $Q_3$, $IQR$을 구하여, 이상치 여부를 판별합니다.

> $Q_1 - 1.5×IQR ≤X≤ Q_3 + 1.5×IQR$

이중에 하나라도 이상치라면 제거하여 df_berry2셋을 만듭니다.
    
</div>

In [11]:
df_iqr = df_berry[['fruitset', 'fruitmass', 'seeds']].apply(
    lambda x: x.quantile([0.25, 0.75]).rename({0.25:'Q1', 0.75:'Q3'})
).T.assign(
    IQR = lambda x: x['Q3'] - x['Q1']
).assign(
    LB = lambda x: x['Q1'] - 1.5 * x['IQR'],
    UB = lambda x: x['Q3'] + 1.5 * x['IQR']
)
df_iqr

,Q1,Q3,IQR,LB,UB
fruitset,0.454725,0.561297,0.106571,0.294868,0.721153
fruitmass,0.416281,0.476149,0.059869,0.326478,0.565952
seeds,33.116091,39.239668,6.123577,23.930726,48.425033


In [15]:
df_berry[['fruitset','fruitmass','seeds']]

,fruitset,fruitmass,seeds
0,0.410652,0.408159,31.678898
1,0.444254,0.425458,33.449385
2,0.383787,0.399172,30.546306
3,0.407564,0.408789,31.562586
4,0.354413,0.382703,28.873714
...,...,...,...
772,0.486815,0.428012,33.447471
773,0.342841,0.377915,28.462005
774,0.404617,0.401670,30.748240
775,0.401538,0.399935,30.582161


In [22]:
df_is_outlier = df_berry[['fruitset','fruitmass','seeds']].apply(
    lambda x: (x < df_iqr.loc[x.name, 'LB']) | (x > df_iqr.loc[x.name, 'UB'])
)
df_is_outlier.head(10)

,fruitset,fruitmass,seeds
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
6,True,False,False
7,True,False,False
8,False,False,False
9,False,False,False


In [30]:
df_berry.loc[df_is_outlier.any(axis=1), ['fruitset', 'fruitmass', 'seeds']]

,fruitset,fruitmass,seeds
6,0.284443,0.352186,26.101179
7,0.246568,0.342826,25.042361
16,0.288159,0.359459,26.487322
17,0.279536,0.354415,26.282356
18,0.233554,0.335339,24.320627
32,0.262140,0.349354,25.433530
47,0.283055,0.355876,26.054692
766,0.192732,0.311921,22.079199
768,0.249335,0.320727,23.412776


In [ ]:
# 이상치에 해당하는 인덱스로, 이상치를 제거한 DataFrame을 df_berry2로 지정합니다.
df_berry2 = df_berry.drop(index=df_berry.loc[df_is_outlier.any(axis=1).index])

- 범위 한정 (clipping)

하한값을 벗어난 경우 하한값으로 대체하고

상한값을 벗어난 경우 상한값으로 대체

(pd.Series.clip 활용)

##### Q5

<div style="border: 1px solid #ddd; padding: 12px; margin-top: 10px;">

df_berry 셋의 fruitset, fruitmass, seeds 가

각 변수별로 $Q_1$, $Q_3$, $IQR$을 구하여, 이상치 여부를 판별을 합니다.

> $Q_1 - 1.5×IQR ≤X≤ Q_3 + 1.5×IQR$

하한값을 벗어났을 경우 하한값으로 대체하고,

상한값을 벗어났을 경우는 상한값으로 대체하여 df_berry3셋을 만듭니다.

</div>

In [27]:
df_iqr

,Q1,Q3,IQR,LB,UB
fruitset,0.454725,0.561297,0.106571,0.294868,0.721153
fruitmass,0.416281,0.476149,0.059869,0.326478,0.565952
seeds,33.116091,39.239668,6.123577,23.930726,48.425033


In [32]:
df_berry3 = df_berry.copy()

In [34]:
df_berry3[['fruitset','fruitmass', 'seeds']] = df_berry3[['fruitset', 'fruitmass', 'seeds']].apply(
    lambda x: x.clip(df_iqr.loc[x.name, 'LB'], df_iqr.loc[x.name, 'UB'])
)

In [39]:
df_is_outlier3 = df_berry3[['fruitset','fruitmass','seeds']].apply(
    lambda x: (x < df_iqr.loc[x.name, 'LB']) | (x > df_iqr.loc[x.name, 'UB'])
)

0      False
1      False
2      False
3      False
4      False
       ...  
772    False
773    False
774    False
775    False
776    False
Length: 777, dtype: bool

In [40]:
df_is_outlier3

,fruitset,fruitmass,seeds
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
772,False,False,False
773,False,False,False
774,False,False,False
775,False,False,False


In [42]:
len(df_is_outlier3.loc[df_is_outlier3.any(axis=1)])

0